In [20]:
# read VC7

In [21]:
# If you don't have it installed
#!pip install readim

In [4]:
import ReadIM
import os, glob

In [5]:
!ls ../data/davis_vc7/

test_file_2D = '../data/davis_vc7/2C.VC7'
test_file_3D = '../data/davis_vc7/2C.VC7'

2C.VC7 3C.VC7


In [36]:
# from ReadIM readme: 
vbuff, vatts = ReadIM.extra.get_Buffer_andAttributeList(test_file_2D)
v_array, vbuff = ReadIM.extra.buffer_as_array(vbuff)

In [41]:
def read_VC7(filename):
    """
    Return the buffers stored in the given file.
    """
    vbuff, vatts = ReadIM.extra.get_Buffer_andAttributeList(filename)
    v_array, vbuff2 = ReadIM.extra.buffer_as_array(vbuff)
    v_array = np.array(v_array.transpose((0, 2, 1)))
    vatts = ReadIM.extra.att2dict(vatts)
    fmt = vbuff.image_sub_type
    vectorGrid = vbuff.vectorGrid
    ReadIM.DestroyBuffer(vbuff)
    ReadIM.DestroyBuffer(vbuff2)
    return fmt, vectorGrid, v_array, vatts


In [42]:
read_VC7(test_file_2D)

(3, 24, array([[[ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  5.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  5.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  5.00000000e+00],
         ...,
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  0.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
 
        [[-1.95647550e+00,  0.00000000e+00, -7.96253586e+00, ...,
          -4.81630278e+00,  0.00000000e+00,  0.00000000e+00],
         [-8.59071255e+00, -1.78645873e+00, -1.65714705e+00, ...,
          -2.6

In [43]:
read_VC7(test_file_3D)

(3, 24, array([[[ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  5.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  5.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  5.00000000e+00],
         ...,
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           5.00000000e+00,  5.00000000e+00,  0.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
 
        [[-1.95647550e+00,  0.00000000e+00, -7.96253586e+00, ...,
          -4.81630278e+00,  0.00000000e+00,  0.00000000e+00],
         [-8.59071255e+00, -1.78645873e+00, -1.65714705e+00, ...,
          -2.6

In [8]:
def import_from_VC7(filename, infos=False, add_fields=False):
    """
    Import a vector field or a velocity field from a .VC7 file

    Parameters
    ----------
    filename : string
        Path to the file to import.
    infos : boolean, optional
        If 'True', also return a dictionary with informations on the im7
    add_fields : boolean, optional
        If 'True', also return a tuple containing additional fields
        contained in the vc7 field (peak ratio, correlation value, ...)
    """
    fmt, vectorGrid, v_array, atts = read_VC7(filename)
    # Values and Mask
    if fmt == 2:
        Vx = v_array[0]
        Vy = v_array[1]
        mask = np.zeros(Vx.shape, dtype=bool)
    elif fmt == 3 or fmt == 1:
        mask = np.logical_not(v_array[0])
        mask2 = np.logical_not(v_array[9])
        mask = np.logical_or(mask, mask2)
        Vx = v_array[1]
        Vy = v_array[2]
    mask = np.logical_or(mask, np.logical_and(Vx == 0., Vy == 0.))
    # additional fields if necessary
    if add_fields and fmt in [1, 3]:
        suppl_fields = []
        for i in np.arange(4, v_array.shape[0]):
            suppl_fields.append(np.transpose(np.array(v_array[i])))
    # Get and apply scale on values
    scale_i = atts['_SCALE_I']
    scale_i = scale_i.split("\n")
    unit_values = scale_i[1]
    scale_val = scale_i[0].split(' ')
    try:
        Vx *= int(scale_val[0])
        Vx += int(scale_val[1])
        Vy *= int(scale_val[0])
        Vy += int(scale_val[1])
    except ValueError:
        Vx *= float(scale_val[0])
        Vx += float(scale_val[1])
        Vy *= float(scale_val[0])
        Vy += float(scale_val[1])
    # Get and apply scale on X
    scale_x = atts['_SCALE_X']
    scale_x = scale_x.split("\n")
    unit_x = scale_x[1]
    scale_val = scale_x[0].split(' ')
    x_init = float(scale_val[1])
    dx = float(scale_val[0])*vectorGrid
    len_axe_x = Vx.shape[0]
    if dx < 0:
        axe_x = x_init + np.arange(len_axe_x - 1, -1, -1)*dx
        Vx = -Vx[::-1, :]
        Vy = Vy[::-1, :]
        mask = mask[::-1, :]
        if add_fields:
            for i in np.arange(len(suppl_fields)):
                suppl_fields[i] = suppl_fields[i][::-1, :]
    else:
        axe_x = x_init + np.arange(len_axe_x)*dx
    # Get and apply scale on Y
    scale_y = atts['_SCALE_Y']
    scale_y = scale_y.split("\n")
    unit_y = scale_y[1]
    scale_val = scale_y[0].split(' ')
    y_init = float(scale_val[1])
    dy = float(scale_val[0])*vectorGrid
    len_axe_y = Vx.shape[1]
    if dy < 0 or scale_y[1] == 'pixel':
        axe_y = y_init + np.arange(len_axe_y - 1, -1, -1)*dy
        Vx = Vx[:, ::-1]
        Vy = -Vy[:, ::-1]
        mask = mask[:, ::-1]
        if add_fields:
            for i in np.arange(len(suppl_fields)):
                suppl_fields[i] = suppl_fields[i][:, ::-1]
    else:
        axe_y = y_init + np.arange(len_axe_y)*dy
        
    return axe_x,axe_y,Vx,Vy,mask,unit_x,unit_y,unit_values


In [12]:
import_from_VC7(test_file_2D)

(array([-146.610992, -140.594624, -134.578256, -128.561888, -122.54552 ,
        -116.529152, -110.512784, -104.496416,  -98.480048,  -92.46368 ,
         -86.447312,  -80.430944,  -74.414576,  -68.398208,  -62.38184 ,
         -56.365472,  -50.349104,  -44.332736,  -38.316368,  -32.3     ,
         -26.283632,  -20.267264,  -14.250896,   -8.234528,   -2.21816 ,
           3.798208,    9.814576,   15.830944,   21.847312,   27.86368 ,
          33.880048,   39.896416,   45.912784,   51.929152,   57.94552 ,
          63.961888,   69.978256,   75.994624,   82.010992,   88.02736 ,
          94.043728,  100.060096,  106.076464,  112.092832,  118.1092  ,
         124.125568,  130.141936,  136.158304,  142.174672,  148.19104 ,
         154.207408,  160.223776,  166.240144,  172.256512,  178.27288 ,
         184.289248,  190.305616]),
 array([-321.826258, -315.80989 , -309.793522, -303.777154, -297.760786,
        -291.744418, -285.72805 , -279.711682, -273.695314, -267.678946,
        -261.66

In [13]:
import_from_VC7(test_file_3D)

(array([-146.610992, -140.594624, -134.578256, -128.561888, -122.54552 ,
        -116.529152, -110.512784, -104.496416,  -98.480048,  -92.46368 ,
         -86.447312,  -80.430944,  -74.414576,  -68.398208,  -62.38184 ,
         -56.365472,  -50.349104,  -44.332736,  -38.316368,  -32.3     ,
         -26.283632,  -20.267264,  -14.250896,   -8.234528,   -2.21816 ,
           3.798208,    9.814576,   15.830944,   21.847312,   27.86368 ,
          33.880048,   39.896416,   45.912784,   51.929152,   57.94552 ,
          63.961888,   69.978256,   75.994624,   82.010992,   88.02736 ,
          94.043728,  100.060096,  106.076464,  112.092832,  118.1092  ,
         124.125568,  130.141936,  136.158304,  142.174672,  148.19104 ,
         154.207408,  160.223776,  166.240144,  172.256512,  178.27288 ,
         184.289248,  190.305616]),
 array([-321.826258, -315.80989 , -309.793522, -303.777154, -297.760786,
        -291.744418, -285.72805 , -279.711682, -273.695314, -267.678946,
        -261.66

In [1]:
# either install IM or look into its reading procedure
# https://bitbucket.org/fleming79/im/src/master/IM/core.py